# IMPORTS

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier

## LEITURA DOS DADOS

In [3]:
df = pd.read_csv('/home/cid34senhas/Desktop/DAATP/RawData/train_radiomics_hipocamp.csv') 
df_test = pd.read_csv('/home/cid34senhas/Desktop/DAATP/RawData/test_radiomics_hipocamp.csv')


print(df['Transition'])

0        CN-CN
1        CN-CN
2        AD-AD
3       CN-MCI
4        CN-CN
        ...   
300      CN-CN
301      CN-CN
302     MCI-AD
303    MCI-MCI
304      CN-CN
Name: Transition, Length: 305, dtype: object


# Tratamento

### Single Value Colums 

In [4]:
df = df.loc[:, df.nunique() > 1]
df_test = df_test.loc[:, df_test.nunique() > 1]

#  Analise das colunas que tem menos de 50 valores unicos 
n = df.nunique()
#for col, e in n.items():
#    if e < 50:  
#        print(f"Coluna: {col}, Valores Unicos : {e}")

### AGE BINING  ( secalhar isto era no 2 )

In [5]:

age_bins = [0, 65, 75, 85, 100]
# BINS_SIZER = ['<65', '65-74', '75-84', '85+']
age_labels = [60, 70, 80, 90] # VALOR MEDIO DO BIN 
df['Age'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels).astype(int)
df_test['Age'] = pd.cut(df_test['Age'], bins=age_bins, labels=age_labels).astype(int)

#print(df['Age'])



### Colunas Categoricas 

In [6]:
# Analisar a contagem de valores únicos para cada coluna categórica

# Identificar as colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#for col in categorical_columns:
#    print(f"\nColuna: {col}")

#colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] 

colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'] 

# ** Bounding Box
#
# ** as colunas do 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'
# ** Deveriam ser retiradas, mas o bounding box pode ser importante para a zona de maior ativação do Alzimeir 
# **  ja a de centro de maxima devem ser muito correlacionados, por isso devem ser retirados mais para a frente 
# *TODO acabei por retirar para correr melhor os modelos, mas analisar se se deve retirar ou nao 

df.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)
df_test.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)

### Eliminar Transition CN-MCI 

Como este tipo de precisao não vai ser realiza, decidimos retirar todas as linhas com ela relacinadas para nao confundir o nosso modelo

In [7]:
# df = df[df['Transition'] != 'CN-MCI']

### Label Encoding

In [8]:
label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}


# Apply the mapping to the target column
#df['Transition'] = df['Transition'].map(label_mapping)

## Models

In [9]:

#experiment = setup(df, target='Transition')

In [10]:
#best = compare_models()

Com o tratamento atual, o pycaret diz que os melhores modelos são :
- Random Forest Classifier
- Extra Trees Classifier
- Gradient Boosting Classifier 
- K Neighbors Classifier
- Extreme Gradient Boosting 


### Aplicação dos modelos (default) ao dataset tratado

In [20]:
print(df['Transition'].dtype)
print(df['Transition'])

df['Transition'] = df['Transition'].astype(str)

df[['Origem', 'Destino']] = df['Transition'].str.split('-', expand=True)
print(df[['Origem', 'Destino', 'Transition']])



object
0        CN-CN
1        CN-CN
2        AD-AD
3       CN-MCI
4        CN-CN
        ...   
300      CN-CN
301      CN-CN
302     MCI-AD
303    MCI-MCI
304      CN-CN
Name: Transition, Length: 305, dtype: object
    Origem Destino Transition
0       CN      CN      CN-CN
1       CN      CN      CN-CN
2       AD      AD      AD-AD
3       CN     MCI     CN-MCI
4       CN      CN      CN-CN
..     ...     ...        ...
300     CN      CN      CN-CN
301     CN      CN      CN-CN
302    MCI      AD     MCI-AD
303    MCI     MCI    MCI-MCI
304     CN      CN      CN-CN

[305 rows x 3 columns]


In [12]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

In [13]:
def gerar_transition_previsao(row, stats):
    # Combinações válidas
    combinacoes_validas = {'CN-CN', 'AD-AD', 'MCI-AD', 'MCI-MCI', 'CN-MCI'}

    # Criar a combinação da origem e destino previstos
    transition = f"{row['Origem_Prevista']}-{row['Destino_Prevista']}"

    # Verificar se a combinação é válida
    if transition in combinacoes_validas:
        return transition
    else:
        # Regras para combinações inválidas
        if row['Destino_Prevista'] == 'MCI' and row['Origem_Prevista'] not in {'MCI', 'CN'}:
            stats['alterados'] += 1
            print(" row['Destino_Prevista'] == 'MCI' and row['Origem_Prevista'] not in {'MCI', 'CN'}:")
            return 'AD-AD'
        elif row['Origem_Prevista'] == 'CN' and row['Destino_Prevista'] not in {'MCI', 'CN'}:
            stats['alterados'] += 1
            print(" row['Origem_Prevista'] == 'CN' and row['Destino_Prevista'] not in {'MCI', 'CN'}:")
            return 'CN-MCI'
        elif row['Destino_Prevista'] == 'AD' and row['Origem_Prevista'] != 'AD':
            stats['alterados'] += 1
            print("row['Destino_Prevista'] == 'AD' and row['Origem_Prevista'] != 'AD':")
            return 'AD-AD'
        elif row['Origem_Prevista'] == 'MCI' and row['Destino_Prevista'] not in {'MCI', 'AD'}:
            stats['alterados'] += 1
            print("        elif row['Origem_Prevista'] == 'MCI' and row['Destino_Prevista'] not in {'MCI', 'AD'}:")
            return 'MCI-MCI'
        else:
            # Contar os casos que não foram corrigidos por falta de regra
            stats['nao_corrigidos'] += 1
            return transition  # Retorna o original, mesmo que inválido


In [14]:
def processar_transicoes(df, modelo_origem, modelo_destino): 
    # Separar as features e os targets
    X = df.drop(columns=['Origem', 'Destino', 'Transition'])
    y_origem = df['Origem']
    y_destino = df['Destino']
    y_transtion = df['Transition']
    # Dividir o conjunto de dados em treino e teste para validação

    # Divisão principal para garantir que os índices de teste sejam os mesmos
    X_train_transition, X_test_transition, y_train_transition, y_test_transition = train_test_split(
        X, y_transtion, test_size=0.40, random_state=34, stratify=y_transtion
    )

    # Usar os mesmos índices de treino e teste para y_origem e y_destino
    y_train_origem, y_test_origem = y_origem[X_train_transition.index], y_origem[X_test_transition.index]
    y_train_destino, y_test_destino = y_destino[X_train_transition.index], y_destino[X_test_transition.index]

    X_train_origem, X_test_origem = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    X_train_destino, X_test_destino = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    #


    # ? ORIGEM #########################################################
    # Criar o modelo Random Forest
    rf_origem_model = modelo_origem
    # Avaliar desempenho com cross-validation no treino
    rf_score_origem = cross_val_score(rf_origem_model, X_train_origem, y_train_origem, cv=5, scoring='f1_macro')
    print("F1 Macro (Cross-Validation):", rf_score_origem.mean())
    # Treinar o modelo no conjunto de treino e prever no conjunto de teste
    rf_origem_model.fit(X_train_origem, y_train_origem)
    rf_origem_pred = rf_origem_model.predict(X_test_origem)
    # Avaliar o desempenho no conjunto de teste
    print("Relatório de Classificação (Teste):")
    print(classification_report(y_test_origem, rf_origem_pred))



    # ? Destino #########################################################
    # Criar o modelo Random Forest
    rf_destino_model = modelo_destino
    # Avaliar desempenho com cross-validation no treino
    rf_score_destino = cross_val_score(rf_destino_model, X_train_destino, y_train_destino, cv=5, scoring='f1_macro')
    print("F1 Macro (Cross-Validation):", rf_score_destino.mean())
    # Treinar o modelo no conjunto de treino e prever no conjunto de teste
    rf_destino_model.fit(X_train_destino, y_train_destino)
    rf_destino_pred = rf_destino_model.predict(X_test_destino)
    # Avaliar o desempenho no conjunto de teste
    print("Relatório de Classificação (Teste):")
    print(classification_report(y_test_destino, rf_destino_pred))



    # Fazer previsões para o conjunto de teste
    origem_pred_test = rf_origem_model.predict(X_test_transition)
    destino_pred_test = rf_destino_model.predict(X_test_transition)

    # Criar um DataFrame com as previsões do conjunto de teste
    Previsoes_teste = pd.DataFrame({
        'Origem_Prevista': origem_pred_test,
        'Destino_Prevista': destino_pred_test
    })

    # Inicializar o dicionário para armazenar estatísticas
    stats_test = {'alterados': 0, 'nao_corrigidos': 0}

    # Gerar a coluna Transition-Previsao para o conjunto de teste
    Previsoes_teste['Transition-Previsao'] = Previsoes_teste.apply(
        gerar_transition_previsao, axis=1, stats=stats_test
    )

    # Comparar as previsões no conjunto de teste
    print("Relatório de Classificação (Teste):")
    print(classification_report(y_test_transition, Previsoes_teste['Transition-Previsao']))

    # Exibir contagens de alterações e casos não corrigidos
    print(f"Elementos alterados (teste): {stats_test['alterados']}")
    print(f"Casos não corrigidos (teste): {stats_test['nao_corrigidos']}")

    res =  Previsoes_teste['Transition-Previsao'] 

    report_dict = classification_report(
    y_test_transition, 
    Previsoes_teste['Transition-Previsao'], 
    output_dict=True)


    return Previsoes_teste['Transition-Previsao'], report_dict





In [15]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

def processar_transicoes_com_label_encoder(df, modelo_origem, modelo_destino):
    # Separar as features e os targets
    X = df.drop(columns=['Origem', 'Destino', 'Transition'])
    y_origem = df['Origem']
    y_destino = df['Destino']
    y_transition = df['Transition']

    # Codificar as classes para y_origem, y_destino e y_transition
    label_encoder_origem = LabelEncoder()
    y_origem_encoded = label_encoder_origem.fit_transform(y_origem)

    label_encoder_destino = LabelEncoder()
    y_destino_encoded = label_encoder_destino.fit_transform(y_destino)

    label_encoder_transition = LabelEncoder()
    y_transition_encoded = label_encoder_transition.fit_transform(y_transition)

    # Dividir o conjunto de dados em treino e teste para validação
    X_train_transition, X_test_transition, y_train_transition, y_test_transition = train_test_split(
        X, y_transition_encoded, test_size=0.40, random_state=34, stratify=y_transition_encoded
    )

    # Usar os mesmos índices de treino e teste para y_origem e y_destino
    y_train_origem, y_test_origem = y_origem_encoded[X_train_transition.index], y_origem_encoded[X_test_transition.index]
    y_train_destino, y_test_destino = y_destino_encoded[X_train_transition.index], y_destino_encoded[X_test_transition.index]

    X_train_origem, X_test_origem = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    X_train_destino, X_test_destino = X.loc[X_train_transition.index], X.loc[X_test_transition.index]

    # Treinar e avaliar o modelo de Origem
    rf_score_origem = cross_val_score(modelo_origem, X_train_origem, y_train_origem, cv=5, scoring='f1_macro')
    print("F1 Macro (Cross-Validation - Origem):", rf_score_origem.mean())
    print("Desvio Padrão F1 Macro (Origem):", rf_score_origem.std())

    modelo_origem.fit(X_train_origem, y_train_origem)
    origem_pred = modelo_origem.predict(X_test_origem)

    # Treinar e avaliar o modelo de Destino
    rf_score_destino = cross_val_score(modelo_destino, X_train_destino, y_train_destino, cv=5, scoring='f1_macro')
    print("F1 Macro (Cross-Validation - Destino):", rf_score_destino.mean())
    print("Desvio Padrão F1 Macro (Destino):", rf_score_destino.std())

    modelo_destino.fit(X_train_destino, y_train_destino)
    destino_pred = modelo_destino.predict(X_test_destino)

    # Fazer previsões para o conjunto de teste de Transition
    origem_pred_test = modelo_origem.predict(X_test_transition)
    destino_pred_test = modelo_destino.predict(X_test_transition)

    # Decodificar as previsões para os valores originais
    origem_pred_decoded = label_encoder_origem.inverse_transform(origem_pred_test)
    destino_pred_decoded = label_encoder_destino.inverse_transform(destino_pred_test)

    # Criar DataFrame de previsões
    Previsoes_teste = pd.DataFrame({
        'Origem_Prevista': origem_pred_decoded,
        'Destino_Prevista': destino_pred_decoded
    })

    # Inicializar o dicionário de estatísticas
    stats_test = {'alterados': 0, 'nao_corrigidos': 0}

    # Gerar a coluna Transition-Previsao
    Previsoes_teste['Transition-Previsao'] = Previsoes_teste.apply(
        gerar_transition_previsao, axis=1, stats=stats_test
    )

    # Avaliar desempenho no conjunto de teste
    y_test_transition_decoded = label_encoder_transition.inverse_transform(y_test_transition)
    print("Relatório de Classificação (Teste):")
    print(classification_report(y_test_transition_decoded, Previsoes_teste['Transition-Previsao']))
    
    print(f"Elementos alterados (teste): {stats_test['alterados']}")
    print(f"Casos não corrigidos (teste): {stats_test['nao_corrigidos']}")

    report_dict = classification_report(
    y_test_transition_decoded, 
    Previsoes_teste['Transition-Previsao'], 
    output_dict=True
)
    return Previsoes_teste , report_dict


In [16]:
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier

def processar_transicoes_com_label_encoder_reducao_features(df, modelo_origem, modelo_destino):
    # Separar as features e os targets
    X = df.drop(columns=['Origem', 'Destino', 'Transition'])
    y_origem = df['Origem']
    y_destino = df['Destino']
    y_transition = df['Transition']

    # Codificar as classes para y_origem, y_destino e y_transition
    label_encoder_origem = LabelEncoder()
    y_origem_encoded = label_encoder_origem.fit_transform(y_origem)

    label_encoder_destino = LabelEncoder()
    y_destino_encoded = label_encoder_destino.fit_transform(y_destino)

    label_encoder_transition = LabelEncoder()
    y_transition_encoded = label_encoder_transition.fit_transform(y_transition)

    # Dividir o conjunto de dados em treino e teste para validação
    X_train_transition, X_test_transition, y_train_transition, y_test_transition = train_test_split(
        X, y_transition_encoded, test_size=0.40, random_state=34, stratify=y_transition_encoded
    )

    # Usar os mesmos índices de treino e teste para y_origem e y_destino
    y_train_origem, y_test_origem = y_origem_encoded[X_train_transition.index], y_origem_encoded[X_test_transition.index]
    y_train_destino, y_test_destino = y_destino_encoded[X_train_transition.index], y_destino_encoded[X_test_transition.index]

    X_train_origem, X_test_origem = X.loc[X_train_transition.index], X.loc[X_test_transition.index]
    X_train_destino, X_test_destino = X.loc[X_train_transition.index], X.loc[X_test_transition.index]



    modelo_origem.fit(X_train_origem, y_train_origem)
    origem_pred = modelo_origem.predict(X_test_origem)

##########
    # Aplicar SelectFromModel para reduzir as features
    selector = SelectFromModel(estimator=modelo_origem)
    X_train_reduzido = selector.fit_transform(X_train_origem, y_train_origem)
    X_test_reduzido = selector.transform(X_test_transition)
    
    # Exibir as dimensões antes e depois da redução
    print("Dimensão original:", X_train_origem.shape)
    print("Dimensão reduzida:", X_train_reduzido.shape)
    
    # Treinar o modelo com as features reduzidas
    modelo_reduzido = modelo_origem
    modelo_reduzido.fit(X_train_reduzido, y_train_origem)
    y_pred_reduzido_origem = modelo_reduzido.predict(X_test_reduzido)


        # Treinar e avaliar o modelo de Origem
    rf_score_origem = cross_val_score(modelo_reduzido, X_train_reduzido, y_train_origem, cv=5, scoring='f1_macro')
    print("F1 Macro (Cross-Validation - Origem):", rf_score_origem.mean())
    print("Desvio Padrão F1 Macro (Origem):", rf_score_origem.std())
############



    modelo_destino.fit(X_train_destino, y_train_destino)
    destino_pred = modelo_destino.predict(X_test_destino)

    # Aplicar SelectFromModel para reduzir as features
    selector = SelectFromModel(modelo_destino, threshold="median")
    X_train_reduzido = selector.fit_transform(X_train_destino, y_train_destino)
    X_test_reduzido = selector.transform(X_test_transition)
    
    # Exibir as dimensões antes e depois da redução
    print("Dimensão original:", X_train_destino.shape)
    print("Dimensão reduzida:", X_train_reduzido.shape)
    
    # Treinar o modelo com as features reduzidas
    modelo_reduzido =modelo_destino
    modelo_reduzido.fit(X_train_reduzido, y_train_destino)
    y_pred_reduzido = modelo_reduzido.predict(X_test_reduzido)



    # Treinar e avaliar o modelo de Destino
    rf_score_destino = cross_val_score(modelo_reduzido, X_train_reduzido, y_train_destino, cv=5, scoring='f1_macro')
    print("F1 Macro (Cross-Validation - Destino):", rf_score_destino.mean())
    print("Desvio Padrão F1 Macro (Destino):", rf_score_destino.std())
################
    # Fazer previsões para o conjunto de teste de Transition
    #origem_pred_test = modelo_origem.predict(X_test_transition)
    #destino_pred_test = modelo_destino.predict(X_test_transition)

    # Decodificar as previsões para os valores originais
    origem_pred_decoded = label_encoder_origem.inverse_transform(y_pred_reduzido_origem)
    destino_pred_decoded = label_encoder_destino.inverse_transform(y_pred_reduzido)

    # Criar DataFrame de previsões
    Previsoes_teste = pd.DataFrame({
        'Origem_Prevista': origem_pred_decoded,
        'Destino_Prevista': destino_pred_decoded
    })

    # Inicializar o dicionário de estatísticas
    stats_test = {'alterados': 0, 'nao_corrigidos': 0}

    # Gerar a coluna Transition-Previsao
    Previsoes_teste['Transition-Previsao'] = Previsoes_teste.apply(
        gerar_transition_previsao, axis=1, stats=stats_test
    )

    # Avaliar desempenho no conjunto de teste
    y_test_transition_decoded = label_encoder_transition.inverse_transform(y_test_transition)
    print("Relatório de Classificação (Teste):")
    print(classification_report(y_test_transition_decoded, Previsoes_teste['Transition-Previsao']))
    
    print(f"Elementos alterados (teste): {stats_test['alterados']}")
    print(f"Casos não corrigidos (teste): {stats_test['nao_corrigidos']}")

    report_dict = classification_report(
    y_test_transition_decoded, 
    Previsoes_teste['Transition-Previsao'], 
    output_dict=True
)
    return Previsoes_teste , report_dict


In [17]:
#from xgboost import XGBClassifier
#
## Criar instâncias dos modelos
#xgbost_origem = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)
#xgbost_destino = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.05,colsample_bytree=0.4, subsample=0.8, random_state=34)
#
random_forest_origem = RandomForestClassifier(n_estimators=800,random_state=34)
random_forest_destino = RandomForestClassifier(n_estimators=800,random_state=34)
processar_transicoes(df,random_forest_origem,random_forest_destino)
#
## Chamar a função com as instâncias dos modelos
#previsoes_xgboosd, report_classificacao_xgboost = processar_transicoes_com_label_encoder(df, xgbost_origem, xgbost_destino)
#
#previsoes_random_forest, report_classificacao_random_forest = processar_transicoes(df, random_forest_origem,random_forest_destino)
##grandient_origem  = GradientBoostingClassifier(
##        learning_rate=0.1, n_estimators=100, random_state=34
##    )
##extra_trees_destino =  ExtraTreesClassifier(
##        n_estimators=447, max_features='sqrt', max_depth=15,
##        min_samples_split=10, min_samples_leaf=5, bootstrap=False, random_state=34
##    )
##previsoes, report_classificacao_xgboost = processar_transicoes_com_label_encoder_reducao_features(df, grandient_origem, grandient_origem)

F1 Macro (Cross-Validation): 0.5187944570049834
Relatório de Classificação (Teste):
              precision    recall  f1-score   support

          AD       0.50      0.33      0.40        24
          CN       0.66      0.79      0.72        42
         MCI       0.57      0.57      0.57        56

    accuracy                           0.60       122
   macro avg       0.58      0.56      0.56       122
weighted avg       0.59      0.60      0.59       122

F1 Macro (Cross-Validation): 0.4843218957796963
Relatório de Classificação (Teste):
              precision    recall  f1-score   support

          AD       0.74      0.82      0.78        51
          CN       0.56      0.76      0.64        38
         MCI       0.38      0.15      0.22        33

    accuracy                           0.62       122
   macro avg       0.56      0.58      0.55       122
weighted avg       0.59      0.62      0.58       122

        elif row['Origem_Prevista'] == 'MCI' and row['Destino_Prevista

(0      MCI-AD
 1       CN-CN
 2      MCI-AD
 3      MCI-AD
 4      MCI-AD
         ...  
 117    MCI-AD
 118    MCI-AD
 119     AD-AD
 120    MCI-AD
 121    MCI-AD
 Name: Transition-Previsao, Length: 122, dtype: object,
 {'AD-AD': {'precision': 0.5,
   'recall': 0.3333333333333333,
   'f1-score': 0.4,
   'support': 24.0},
  'CN-CN': {'precision': 0.6041666666666666,
   'recall': 0.7631578947368421,
   'f1-score': 0.6744186046511628,
   'support': 38.0},
  'CN-MCI': {'precision': 0.5,
   'recall': 0.25,
   'f1-score': 0.3333333333333333,
   'support': 4.0},
  'MCI-AD': {'precision': 0.2926829268292683,
   'recall': 0.4444444444444444,
   'f1-score': 0.35294117647058826,
   'support': 27.0},
  'MCI-MCI': {'precision': 0.4,
   'recall': 0.20689655172413793,
   'f1-score': 0.2727272727272727,
   'support': 29.0},
  'accuracy': 0.45901639344262296,
  'macro avg': {'precision': 0.459369918699187,
   'recall': 0.39956644484775156,
   'f1-score': 0.4066840774364714,
   'support': 122.0},
  'w

In [18]:
from sklearn.base import ClassifierMixin
import pandas as pd
from sklearn.metrics import classification_report

def generate_predictions_csv(
    origem_model: ClassifierMixin, 
    destino_model: ClassifierMixin, 
    test_data_set: pd.DataFrame, 
):
    """
    Gera um arquivo CSV com as previsões combinadas de dois modelos fornecidos e o conjunto de teste.
    Inclui estatísticas de transições corrigidas e não corrigidas.

    Parameters:
    - origem_model: Modelo treinado para prever a origem.
    - destino_model: Modelo treinado para prever o destino.
    - test_data_set: DataFrame com os dados de teste.
    """
    # Fazer previsões para o conjunto de teste
    origem_pred_test = origem_model.predict(test_data_set)
    destino_pred_test = destino_model.predict(test_data_set)

    # Criar um DataFrame com as previsões do conjunto de teste
    Previsoes_teste = pd.DataFrame({
        'Origem_Prevista': origem_pred_test,
        'Destino_Prevista': destino_pred_test
    })

    # Inicializar o dicionário para armazenar estatísticas
    stats_test = {'alterados': 0, 'nao_corrigidos': 0}

    # Gerar a coluna Transition-Previsao para o conjunto de teste
    Previsoes_teste['Transition-Previsao'] = Previsoes_teste.apply(
        gerar_transition_previsao, axis=1, stats=stats_test
    )

    # Renomear a coluna 'Transition-Previsao' para 'Result'
    Previsoes_teste.rename(columns={'Transition-Previsao': 'Result'}, inplace=True)

    # Exibir contagens de alterações e casos não corrigidos
    print(f"Elementos alterados (teste): {stats_test['alterados']}")
    print(f"Casos não corrigidos (teste): {stats_test['nao_corrigidos']}")

    # Salvar o DataFrame em um arquivo CSV
    Previsoes_teste.index = range(1, len(Previsoes_teste) + 1)
    Previsoes_teste.index.name = 'RowId'
    output_filename = f"{type(origem_model).__name__}_{type(destino_model).__name__}_predictions.csv"
    Previsoes_teste.to_csv(output_filename, columns=['Result'], index=True)

    print(f"Resultados salvos no arquivo {output_filename}")




#generate_predictions_csv(modelo_origem,modelo_destino,df_test)

In [19]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pandas as pd


from sklearn.preprocessing import LabelEncoder
import pandas as pd


def generate_predictions_csv_com_label_enconder(
    origem_model, 
    destino_model, 
    test_data_set: pd.DataFrame
):
    """
    Gera um arquivo CSV com as previsões combinadas de dois modelos fornecidos e o conjunto de teste,
    utilizando LabelEncoder para mapear os rótulos.

    Parameters:
    - origem_model: Modelo treinado para prever a origem.
    - destino_model: Modelo treinado para prever o destino.
    - test_data_set: DataFrame com os dados de teste (apenas features, sem rótulos).
    """
    # Criar LabelEncoders para os rótulos de origem e destino
    label_encoder_origem = LabelEncoder()
    label_encoder_destino = LabelEncoder()

    # Definir os rótulos conhecidos (substitua por seus valores reais)
    known_origem_labels = ['CN', 'AD', 'MCI']  # Exemplo de rótulos possíveis para "Origem"
    known_destino_labels = ['CN', 'AD', 'MCI']  # Exemplo de rótulos possíveis para "Destino"

    # Ajustar os encoders com os rótulos conhecidos
    label_encoder_origem.fit(known_origem_labels)
    label_encoder_destino.fit(known_destino_labels)

    # Fazer previsões usando os modelos
    origem_pred = origem_model.predict(test_data_set)
    destino_pred = destino_model.predict(test_data_set)

    # Decodificar as previsões para rótulos textuais
    origem_pred_decoded = label_encoder_origem.inverse_transform(origem_pred)
    destino_pred_decoded = label_encoder_destino.inverse_transform(destino_pred)

    # Criar um DataFrame com as previsões
    Previsoes_teste = pd.DataFrame({
        'Origem_Prevista': origem_pred_decoded,
        'Destino_Prevista': destino_pred_decoded
    })

    # Gerar coluna combinada (Transition-Previsao)
    stats_test = {'alterados': 0, 'nao_corrigidos': 0}

        # Gerar a coluna Transition-Previsao
    Previsoes_teste['Result'] = Previsoes_teste.apply(
        gerar_transition_previsao, axis=1, stats=stats_test
    )

    print(f"Elementos alterados (teste): {stats_test['alterados']}")
    print(f"Casos não corrigidos (teste): {stats_test['nao_corrigidos']}")
    # Salvar previsões em um arquivo CSV
    Previsoes_teste.index = range(1, len(Previsoes_teste) + 1)
    Previsoes_teste.index.name = 'RowId'
    output_filename = f"{type(origem_model).__name__}_{type(destino_model).__name__}_predictions.csv"
    Previsoes_teste.to_csv(output_filename, columns=[ 'Result'], index=True)

    print(f"Resultados salvos no arquivo {output_filename}")
    return Previsoes_teste

#generate_predictions_csv_com_label_enconder(modelo_origem,modelo_destino,df_test)